## Модуль №1 - Что такое SQLAlchemy?

In [7]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

engine = create_engine('sqlite:///test.db')

metadata = MetaData()

users = Table(
    'users',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
)

metadata.create_all(engine)

In [8]:
with engine.connect() as conn:
    conn.execute(users.insert().values(name='Alice'))
    result = conn.execute(users.select())
    for row in result:
        print(row)

(1, 'Alice')


In [9]:
books = Table(
    'books',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String),
    Column('author', String),
)

metadata.create_all(engine)

In [13]:
with engine.connect() as conn:
    conn.execute(books.insert().values(title='Война и мир', author='Лев Толстой'))
    conn.execute(books.insert().values(title='Тихий Дон', author='Михаил Шолохов'))
    result = conn.execute(books.select())
    for row in result:
        print(row)


(1, 'Война и мир', 'Лев Толстой')
(2, 'Тихий Дон', 'Михаил Шолохов')


## Модуль №2 - SQLAlchemy Core - работа с SQL через Python

In [20]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select, and_, or_

engine = create_engine("sqlite:///books.db", echo=True)

metadata = MetaData()

books = Table(
    'books',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String, nullable=False),
    Column('author', String),
)

metadata.create_all(engine)

with engine.connect() as conn:
    insert_smth = books.insert().values(title='1984', author='George Orwell')
    conn.execute(insert_smth)
    conn.execute(
        books.insert(),
        [
            {"title": "Brave New World", "author": "Aldous Huxley"},
            {"title": "Fahrenheit 451", "author": "Ray Bradbury"},
        ]
    )
    result = conn.execute(select(books))
    for row in result:
        print(row)
    
    # Select specific columns

    select_smth = select(books.c.title, books.c.author)
    result2 = conn.execute(select_smth)

    # Select specific books

    select_smth2 = select(books).where(
        or_(
            books.c.author == 'George Orwell',
            books.c.title.like("%451%")
        )
    )
    result3 = conn.execute(select_smth2)

    # Update the data

    update_smth = books.update().where(books.c.author == "George Orwell").values(title="Nineteen Eighty-Four")
    conn.execute(update_smth)

    delete_smth = books.delete().where(books.c.title == "Brave New World")
    conn.execute(delete_smth)

    result = conn.execute(select(books))
    for row in result:
        print(row)

with engine.begin() as conn:
    conn.execute(books.insert().values(title="Dune", author="Frank Herbert"))
    conn.execute(books.insert().values(title="Hyperion", author="Dan Simmons"))
    

2025-07-23 11:36:19,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 11:36:19,896 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("books")
2025-07-23 11:36:19,896 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-23 11:36:19,897 INFO sqlalchemy.engine.Engine COMMIT
2025-07-23 11:36:19,898 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-23 11:36:19,899 INFO sqlalchemy.engine.Engine INSERT INTO books (title, author) VALUES (?, ?)
2025-07-23 11:36:19,899 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('1984', 'George Orwell')
2025-07-23 11:36:19,900 INFO sqlalchemy.engine.Engine INSERT INTO books (title, author) VALUES (?, ?)
2025-07-23 11:36:19,901 INFO sqlalchemy.engine.Engine [generated in 0.00058s] [('Brave New World', 'Aldous Huxley'), ('Fahrenheit 451', 'Ray Bradbury')]
2025-07-23 11:36:19,902 INFO sqlalchemy.engine.Engine SELECT books.id, books.title, books.author 
FROM books
2025-07-23 11:36:19,902 INFO sqlalchemy.engine.Engine [generated in 0.0003